# clean up, integrate and aggregate Zooplankton data

In [1]:
library(tidyverse, warn.conflicts = FALSE)

# library to read matlab data formats into R
library(reshape2)
library(lubridate)

# set strings as factors to false
options(stringsAsFactors = FALSE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# read Zooplankton data (from csv)

In [2]:
zoo_ds <- read.csv("../DATA/Zooplankton_newest_full.csv")

In [5]:
names(zoo_ds)

[1] "unlimited"         "Cruise"            "Cruise_ID"        
 [4] "Day"               "Month"             "Year"             
 [7] "Date"              "Latitude"          "Longitude"        
[10] "Analyst"           "Mesh_Size"         "TOTAL_DENSITY"    
[13] "BIOMASS"           "ASH"               "COPEPODS"         
[16] "CALANOIDS"         "CYCLOPOIDA"        "HAPARCTICOIDA"    
[19] "POECILOSTOMATOIDA" "L_FISH"            "H_FISH"           
[22] "CHAETOGNATHA"      "CLADOCEROS"        "OSTRACODA"        
[25] "FORAMINIFERA"      "AMPHIPODS"         "ISOPODS"          
[28] "EUPHAUSIIDS"       "MISYDACEA"         "SERGESTID"        
[31] "L_CRUSTACEA"       "N_COPEPOD"         "C_COPEPOD"        
[34] "N_CIRRIPEDIA"      "C_CIRRIPEDIA"      "L_DECAPODA"       
[37] "L_STOMATOPODA"     "L_EUPHAUSIID"      "L_SERGESTID"      
[40] "POLICHAETES"       "L_CYPHONAUTES"     "MEDUSA"           
[43] "SIPHONOPHORES"     "CTENOPHORES"       "L_ECHINODERMS"    
[46] "PTEROPODS"         "HETEROPODS"        "GASTROPODS"       
[49] "L_CEPHALOPODA"     "L_BIVALVE"         "L_GASTROPOD"      
[52] "PROTOZOAN"         "RADIOLARIAN"       "APPENDICULARIANS" 
[55] "SALPS"             "DOLIOLIDS"         "BIVALVES"         
[58] "H_CRUSTACEA"       "L_ANFIOXUS"        "LUCIFER"          
[61] "JUV_GASTROPOD"

In [6]:
str(zoo_ds)

'data.frame':	306 obs. of  61 variables:
 $ unlimited        : int  0 1 2 3 4 5 6 7 8 9 ...
 $ Cruise           : num  71 71 72 72 73 73 74 74 75 75 ...
 $ Cruise_ID        : chr  "CAR-071" "CAR-071" "CAR-072" "CAR-072" ...
 $ Day              : num  9 9 6 6 11 11 10 10 14 14 ...
 $ Month            : num  10 10 11 11 12 12 1 1 2 2 ...
 $ Year             : num  2001 2001 2001 2001 2001 ...
 $ Date             : chr  "2001-10-09" "2001-10-09" "2001-11-06" "2001-11-06" ...
 $ Latitude         : num  10.5 10.5 10.5 10.5 10.5 ...
 $ Longitude        : num  -64.7 -64.7 -64.7 -64.7 -64.7 ...
 $ Analyst          : chr  "Javier Gutierrez" "Javier Gutierrez" "Javier Gutierrez" "Javier Gutierrez" ...
 $ Mesh_Size        : num  200 500 500 200 200 500 200 500 200 500 ...
 $ TOTAL_DENSITY    : num  1112.6 212.9 13.4 282.3 572.6 ...
 $ BIOMASS          : num  26.83 15.95 2.34 7.43 5.6 ...
 $ ASH              : num  6.8428 3.3093 0.0724 0.6393 0.1423 ...
 $ COPEPODS         : num  741.76 160.22 5.6

# remove for outliers in biomass

In [52]:
zoo_ds = zoo_ds[!zoo_ds$BIOMASS >1000,]

# Now get Biomass for Total and the 2 Mesh Sizes Time series

In [53]:
test_zoo <- zoo_ds %>%
    select(BIOMASS, Mesh_Size, Date) %>%
    pivot_wider(names_from=Mesh_Size, values_from=BIOMASS) %>%
    rename("Mesh200" = "200",
          "Mesh500" = "500")

In [54]:
str(test_zoo)


tibble [152 × 3] (S3: tbl_df/tbl/data.frame)
 $ Date   : chr [1:152] "2001-10-09" "2001-11-06" "2001-12-11" "2002-01-10" ...
 $ Mesh200: num [1:152] 26.83 7.43 5.6 10.24 12.92 ...
 $ Mesh500: num [1:152] 15.947 2.337 0.714 4.93 NA ...


In [55]:
zoo_biomass_tot <- test_zoo %>%
    group_by(Date) %>%
    mutate(total_biomass = sum(Mesh200, Mesh500))

In [56]:
zoo_biomass_tot$Mesh200[1]+ zoo_biomass_tot$Mesh500[1]

[1] 42.7806

In [57]:
zoo_biomass_tot$total_biomass

[1] 42.780600  9.765100  6.310324 15.170000        NA 16.020000 17.020000
  [8]  5.810000 15.170000  2.280000  3.070000 13.850000  8.090000  7.420000
 [15] 16.730000 20.540000 33.230000 20.390000 17.290000 22.370000 12.750000
 [22] 13.390000  7.400000 12.990000 13.490000  8.910000 10.930000 28.900000
 [29] 16.580000 24.730000 26.060000 24.660000 14.680000  8.210000  6.400000
 [36]  7.750000  8.950000 16.850000 19.680000 18.360000 16.290000 12.190000
 [43] 11.260000  7.650000 15.140000 25.040000 29.750000 24.840000 24.890000
 [50] 21.970000 12.530000  5.300000 24.100000 23.170000 18.000000  9.570000
 [57] 10.720000 27.170000 17.990000 23.660000 15.370000 26.720000 26.430000
 [64]  6.550000 10.840000 30.270000 30.350000 32.530000 49.440000 22.500000
 [71] 28.940000 14.700000 15.590000 14.200000 15.810000  9.560000  7.130000
 [78] 35.910000 42.860000 26.030000 18.550000 34.570000 25.470000 16.780000
 [85] 28.040000 24.890000 12.160000 13.830000 16.280000 26.316200 11.087100
 [92] 20.537210 26.422590 12.946980 14.481250 14.975850 10.461800  9.417450
 [99]  8.670020  8.215150 11.639750 32.016650 22.801350 19.987520 19.663380
[106] 14.501450 30.232730 27.879800 18.675740 13.996060 10.014020 37.295740
[113] 42.473700 37.436200 62.829700 10.949310 12.240800 13.909700 11.344890
[120] 13.008870        NA 27.905900 29.664600 22.759470 11.659160 16.177880
[127]  7.126310 14.227850  6.698230 20.259720  7.573820 16.853960 21.393350
[134] 47.906300 37.578900 10.683390  8.053430 16.148010 24.626460 12.781050
[141] 17.922050 29.020800 18.528550 10.989900  8.916900 11.035380 18.780540
[148]  8.481240  4.049130 17.141200  5.335940  4.073936

In [61]:
write.csv(zoo_biomass_tot, "ZooplanktonBiomass_CleanedAggregated.csv")